### 문제

* DEVELOPERS 테이블에서 GRADE별 개발자의 정보를 조회하려 합니다. GRADE는 다음과 같이 정해집니다.
* A : Front End 스킬과 Python 스킬을 함께 가지고 있는 개발자
* B : C# 스킬을 가진 개발자
* C : 그 외의 Front End 개발자
* GRADE가 존재하는 개발자의 GRADE, ID, EMAIL을 조회하는 SQL 문을 작성해 주세요.
* 결과는 GRADE와 ID를 기준으로 오름차순 정렬해 주세요.

__MYSQL__ INTUITIVE VERSION

```SQL

WITH BASE AS (
    SELECT 
        T1.ID, 
        T1.EMAIL, 
        GROUP_CONCAT(T2.NAME, ',' , T2.CATEGORY) AS COMBINED
    FROM 
        DEVELOPERS T1
    JOIN 
        SKILLCODES T2
        ON (T1.SKILL_CODE & T2.CODE) = T2.CODE
    GROUP BY
        T1.ID,
        T1.EMAIL
)

SELECT 
    CASE
        WHEN COMBINED LIKE '%Python%' AND  COMBINED LIKE '%Front End%' THEN 'A'
        WHEN COMBINED LIKE '%C#%' THEN 'B'
        WHEN COMBINED LIKE '%Front End%' THEN 'C'
    END AS GRADE,
    ID,
    EMAIL
FROM BASE
HAVING GRADE IS NOT NULL
ORDER BY 
    GRADE ASC,
    ID ASC
;

```

__MYSQL__

```SQL

WITH BASE AS (
    SELECT
        T1.ID,
        T1.EMAIL,
        T2.NAME,
        T2.CATEGORY,
        CASE
            WHEN T2.NAME = 'Python' OR T2.CATEGORY = 'Front End' THEN 1
    END AS MK_A
    FROM DEVELOPERS T1
    JOIN SKILLCODES T2
        ON (T1.SKILL_CODE & T2.CODE) = T2.CODE
),
 A_ID AS (
    SELECT
        ID
    FROM BASE
    GROUP BY
        ID
    HAVING
        SUM(MK_A) >= 2
)

SELECT DISTINCT
    CASE
        WHEN BASE.ID = A_ID.ID THEN 'A'
        WHEN BASE.NAME = 'C#' THEN 'B'
        WHEN BASE.CATEGORY = 'Front End' THEN 'C'
    END AS GRADE,
    BASE.ID,
    BASE.EMAIL
FROM BASE
LEFT JOIN A_ID
    ON BASE.ID = A_ID.ID
HAVING 
    GRADE IS NOT NULL
ORDER BY 
    GRADE ASC,
    ID ASC
;

```